In [ ]:
import matplotlib.pyplot as plt
import numpy as np

 
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier 

plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['figure.figsize'] = (12,6)

%matplotlib inline

In [ ]:
lfw_dataset = fetch_lfw_people(min_faces_per_person=100)

In [ ]:
_, h, w = lfw_dataset.images.shape
X = lfw_dataset.data
y = lfw_dataset.target
target_names = lfw_dataset.target_names
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
n_components = 200
pca = PCA(n_components=n_components).fit(X_train)
 
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(1024,), batch_size=256, verbose=True, early_stopping=True).fit(X_train_pca, y_train)

In [ ]:
y_pred = clf.predict(X_test_pca)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# Visualization
def plot_gallery(images, titles, h, w, rows=3, cols=4):
    plt.figure(figsize=(15,10))
    for i in range(rows * cols):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i])
        plt.xticks(())
        plt.yticks(())

def titles(y_pred, y_test, target_names):
    for i in range(y_pred.shape[0]):
        pred_name = target_names[y_pred[i]].split(' ')[-1]
        true_name = target_names[y_test[i]].split(' ')[-1]
        yield 'predicted: {0}\ntrue: {1}'.format(pred_name, true_name)

prediction_titles = list(titles(y_pred, y_test, target_names))
plot_gallery(X_test, prediction_titles, h, w)

In [ ]:
top_number = 10
top_eigenfaces = pca.components_[:top_number, :].reshape(top_number, h, w)

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(10, 5))
for i in range(10):
    ax[i//5, i%5].imshow(top_eigenfaces[i])

In [ ]:
img_idx = 400

fig, ax = plt.subplots(1, 6, figsize=(12,10))
for i, top_number in enumerate([1, 5, 10, 50, 100, 200]):
    top_eigenfaces = pca.components_[:top_number, :].reshape(top_number, h, w)
    weights = X_train_pca[img_idx][:top_number].reshape(-1, 1, 1)
    eigen_values = pca.components_[:top_number, :].reshape(top_number, h, w)
    reconstruction = (weights * eigen_values).sum(axis=0)
    reconstruction_with_mean = reconstruction + pca.mean_.reshape(h,w)
    ax[i].imshow(reconstruction_with_mean)
    ax[i].set_title(f'{top_number} components')

In [ ]:
#materials from https://pythonmachinelearning.pro/face-recognition-with-eigenfaces/